<font size="+3"><strong>Predicting Price with Neighborhood</strong></font>

# Define Wrangle Function

- Masked dataset for specific category
- Outlier free dataset
- Dataset where a column contains specific string value (Masked)
- Single column expanded to multiple columns
- Dropped duplicate columns

In [1]:
# def wrangle(filepath):
#     # Read CSV file
#     df = pd.read_csv(filepath)

#     # Subset data: Apartments in "Capital Federal", less than 400,000
#     mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
#     mask_apt = df["property_type"] == "apartment"
#     mask_price = df["price_aprox_usd"] < 400_000
#     df = df[mask_ba & mask_apt & mask_price]

#     # Subset data: Remove outliers for "surface_covered_in_m2"
#     low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
#     mask_area = df["surface_covered_in_m2"].between(low, high)
#     df = df[mask_area]

#     # Split "lat-lon" column
#     df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
#     df.drop(columns="lat-lon", inplace=True)

#     return df

# Take all files and Concatenate them

- Ignore index while concatenating / stacking multiple dataframes

In [3]:
# files = glob("data/buenos-aires-real-estate-*.csv")

# frames = []

# for file in files:
#     df = wrangle(file)
#     frames.append(df)

# print(len(frames))
# print(type(frames[0]))

# df = pd.concat(frames, ignore_index = True)
# print(df.shape)
# print(df.info())
# df.head()

# Change Wrangle Function to make specific column from another column

- Take a splitted portion from a column

In [4]:
# print(df.columns)
# def wrangle(filepath):
#     # Read CSV file
#     df = pd.read_csv(filepath)

#     # Subset data: Apartments in "Capital Federal", less than 400,000
#     mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
#     mask_apt = df["property_type"] == "apartment"
#     mask_price = df["price_aprox_usd"] < 400_000
#     df = df[mask_ba & mask_apt & mask_price]

#     # Subset data: Remove outliers for "surface_covered_in_m2"
#     low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
#     mask_area = df["surface_covered_in_m2"].between(low, high)
#     df = df[mask_area]

#     # Split "lat-lon" column
#     df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
#     df.drop(columns="lat-lon", inplace=True)

#     df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand = True)[3]
#     df.drop(columns = ["place_with_parent_names"], inplace = True)

#     return df

# frames = []

# for file in files:
#     df = wrangle(file)
#     frames.append(df)
# df = pd.concat(frames, ignore_index = True)

# Select Training and Target Features 


- Features represent dataframe
- Target represents a single series

In [5]:
# target = "price_aprox_usd"
# features = ["neighborhood"]
# y_train = df[target]
# X_train = df[features]

### Calculate Y for Baseline Model


In [8]:
# y_mean = y_train.mean()
# y_pred_baseline = [y_mean] * len(y_train)
# print("Mean apt price:", y_mean)


# Evaluate Baseline Model


In [1]:
# print("Baseline MAE:", mean_absolute_error(y_train, y_pred_baseline))

# Transform categorical variables in numeric values

- This is a transformation

In [7]:
# print(X_train.head())
# ohe = OneHotEncoder(use_cat_names=True)
# ohe.fit(X_train)
# XT_train = ohe.transform(X_train)
# print(XT_train.shape)
# XT_train.head()

# Make Model Pipeline


In [10]:
# model = make_pipeline(
#     OneHotEncoder(use_cat_names=True),
#     LinearRegression()
# )
# model.fit(X_train, y_train)

### Predict Y for New Model


In [11]:
# y_pred_training = model.predict(X_train)

### Evaluate the New Model

In [13]:
# mae_training = mean_absolute_error(y_train, y_pred_training)
# print("Training MAE:", round(mae_training, 2))


# Make a series out of predicted Y


In [14]:
# X_test = pd.read_csv("data/buenos-aires-test-features.csv")[features]
# y_pred_test = pd.Series(model.predict(X_test))
# y_pred_test.head()

### Take Intercept and Co-efficients and generate equations


In [15]:
# intercept = model.named_steps["linearregression"].intercept_
# coefficients = model.named_steps["linearregression"].coef_
# print("coefficients len:", len(coefficients))
# print(coefficients[:5])  # First five coefficients

# Get Feature Names 

- Works for categorical data and one-hot encoded set

In [16]:
# feature_names = model.named_steps["onehotencoder"].get_feature_names()
# print("features len:", len(feature_names))
# print(feature_names[:5])  # First five feature names

# Get important Features

- Take the co-efficients of the model
- Make the co-efficients into series and index them with feature names

In [17]:
# feat_imp = pd.Series(coefficients, index = feature_names)
# feat_imp.head()

In [18]:
# print(f"price = {intercept.round(2)}")
# for f, c in feat_imp.items():
#     print(f"+ ({round(c, 2)} * {f})")

# Make another Model

In [19]:
# model = make_pipeline(
#     OneHotEncoder(use_cat_names=True),
#     Ridge()
# )
# model.fit(X_train,y_train)

# intercept = model.named_steps["ridge"].intercept_
# coefficients = model.named_steps["ridge"].coef_
# print("coefficients len:", len(coefficients))
# print(coefficients[:5])  # First five coefficients

# Get Feature Names 

In [20]:
# feature_names = model.named_steps["onehotencoder"].get_feature_names()
# print("features len:", len(feature_names))
# print(feature_names[:5])  # First five feature names

# Get important Features

- Take the co-efficients of the model
- Make the co-efficients into series and index them with feature names
- Happens naturally because co-efficients are associated with features in the model 

In [21]:
# feat_imp = pd.Series(coefficients, index = feature_names)
# feat_imp.head()

In [22]:
# print(f"price = {intercept.round(2)}")
# for f, c in feat_imp.items():
#     print(f"+ ({round(c, 2)} * {f})")

# Visualize Feature Importance

- Sort the series
- Remember to sort with absolute values (since importance can be represented in both positive and negative)
- Plot them in horizontal bar chart

In [23]:
# feat_imp.sort_values(key=abs).tail(15).plot(kind = "barh")
# plt.xlabel("Importance [USD]")
# plt.ylabel("Feature")
# plt.title("Feature Importance")